In [131]:
from data_collection.parse_pcap import pcap_to_pandas
from utils import *
import pandas as pd
import numpy as np
from datetime import datetime, timezone
from sklearn.preprocessing import LabelEncoder
from math import log
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
data = pcap_to_pandas("example_pcaps/tplink_switch.pcap")

In [23]:
data

,datetime,dns_query,dns_resp,ip_dst,ip_dst_int,ip_src,ip_src_int,is_dns,length,mac_dst,mac_dst_int,mac_src,mac_src_int,port_dst,port_src,protocol,time,time_normed
0,2017-12-07 15:11:24.315450,None,None,None,NaN,None,NaN,False,113,50:c7:bf:09:f3:4c,88818833814348,b8:27:eb:2d:24:15,202481588839445,NaN,NaN,None,1.512677e+09,0.000000
1,2017-12-07 15:11:24.415702,None,None,None,NaN,None,NaN,False,113,50:c7:bf:09:f3:4c,88818833814348,b8:27:eb:2d:24:15,202481588839445,NaN,NaN,None,1.512677e+09,0.100252
2,2017-12-07 15:11:25.240070,None,None,None,NaN,None,NaN,False,135,b8:27:eb:2d:24:15,202481588839445,50:c7:bf:09:f3:4c,88818833814348,NaN,NaN,None,1.512677e+09,0.924620
3,2017-12-07 15:11:25.241256,None,None,None,NaN,None,NaN,False,169,50:c7:bf:09:f3:4c,88818833814348,b8:27:eb:2d:24:15,202481588839445,NaN,NaN,None,1.512677e+09,0.925806
4,2017-12-07 15:11:25.242138,None,None,None,NaN,None,NaN,False,135,b8:27:eb:2d:24:15,202481588839445,50:c7:bf:09:f3:4c,88818833814348,NaN,NaN,None,1.512677e+09,0.926688
5,2017-12-07 15:11:25.260973,None,None,None,NaN,None,NaN,False,113,b8:27:eb:2d:24:15,202481588839445,50:c7:bf:09:f3:4c,88818833814348,NaN,NaN,None,1.512677e+09,0.945523
6,2017-12-07 15:11:28.529644,None,None,255.255.255.255,4.294967e+09,0.0.0.0,0.000000e+00,False,590,ff:ff:ff:ff:ff:ff,281474976710655,50:c7:bf:09:f3:4c,88818833814348,67.0,68.0,UDP,1.512677e+09,4.214194
7,2017-12-07 15:11:30.246590,None,None,255.255.255.255,4.294967e+09,0.0.0.0,0.000000e+00,False,590,ff:ff:ff:ff:ff:ff,281474976710655,50:c7:bf:09:f3:4c,88818833814348,67.0,68.0,UDP,1.512677e+09,5.931140
8,2017-12-07 15:11:31.033462,None,None,172.24.1.81,2.887254e+09,172.24.1.1,2.887254e+09,False,62,50:c7:bf:09:f3:4c,88818833814348,b8:27:eb:2d:24:15,202481588839445,NaN,NaN,ICMP,1.512677e+09,6.718012
9,2017-12-07 15:11:31.033633,None,None,172.24.1.81,2.887254e+09,172.24.1.1,2.887254e+09,False,342,50:c7:bf:09:f3:4c,88818833814348,b8:27:eb:2d:24:15,202481588839445,68.0,67.0,UDP,1.512677e+09,6.718183


# Probability

Let's compute the probability that a packet from our capture was a DNS packet. In other words, compute the probability that a packet contained either a DNS query or a DNS response field.

Servers that respond to DNS requests typically operate on port 53.

$P($DNS Query $\cup$ DNS Response$)$

Note: filter_data(data, col, pred) returns a DataFrame of only the packets from _data_ where the value in _col_ satisfies _pred_, where _pred_ is any Python code that returns a boolean. Examples:

filter_data(data, "length", "== 62") -> Returns only packets with a length of 62.
filter_data(data, "dns_query", ".notnull()") -> Returns only packets which have a valid DNS query field.

Note: You can use .notnull() on a column to filter out rows that contain "None" or "NaN" in that column.

TODO: Change syntax to filter_data(data, col, op, compare_value=None)

In [47]:
filter_data(data, "dns_query", ".notnull()")

,datetime,dns_query,dns_resp,ip_dst,ip_dst_int,ip_src,ip_src_int,is_dns,length,mac_dst,mac_dst_int,mac_src,mac_src_int,port_dst,port_src,protocol,time,time_normed
15,2017-12-07 15:11:31.532799,b's1a.time.edu.cn.',None,172.24.1.1,2.887254e+09,172.24.1.81,2.887254e+09,True,75,b8:27:eb:2d:24:15,202481588839445,50:c7:bf:09:f3:4c,88818833814348,53.0,32835.0,UDP,1.512677e+09,7.217349
16,2017-12-07 15:11:31.763646,b'devs.tplinkcloud.com.',None,172.24.1.1,2.887254e+09,172.24.1.81,2.887254e+09,True,80,b8:27:eb:2d:24:15,202481588839445,50:c7:bf:09:f3:4c,88818833814348,53.0,43866.0,UDP,1.512677e+09,7.448196
17,2017-12-07 15:11:31.775682,b'devs.tplinkcloud.com.',b'devs.tplinkcloud.com.',172.24.1.81,2.887254e+09,172.24.1.1,2.887254e+09,True,533,50:c7:bf:09:f3:4c,88818833814348,b8:27:eb:2d:24:15,202481588839445,43866.0,53.0,UDP,1.512677e+09,7.460232
21,2017-12-07 15:11:31.885528,b's1a.time.edu.cn.',b's1a.time.edu.cn.',172.24.1.81,2.887254e+09,172.24.1.1,2.887254e+09,True,121,50:c7:bf:09:f3:4c,88818833814348,b8:27:eb:2d:24:15,202481588839445,32835.0,53.0,UDP,1.512677e+09,7.570078
57,2017-12-07 15:11:47.922651,b's1b.time.edu.cn.',None,172.24.1.1,2.887254e+09,172.24.1.81,2.887254e+09,True,75,b8:27:eb:2d:24:15,202481588839445,50:c7:bf:09:f3:4c,88818833814348,53.0,39900.0,UDP,1.512678e+09,23.607201
58,2017-12-07 15:11:47.952146,b's1b.time.edu.cn.',b's1b.time.edu.cn.',172.24.1.81,2.887254e+09,172.24.1.1,2.887254e+09,True,91,50:c7:bf:09:f3:4c,88818833814348,b8:27:eb:2d:24:15,202481588839445,39900.0,53.0,UDP,1.512678e+09,23.636696
69,2017-12-07 15:12:03.990789,b'0.cn.pool.ntp.org.',None,172.24.1.1,2.887254e+09,172.24.1.81,2.887254e+09,True,77,b8:27:eb:2d:24:15,202481588839445,50:c7:bf:09:f3:4c,88818833814348,53.0,55754.0,UDP,1.512678e+09,39.675339
70,2017-12-07 15:12:04.028974,b'0.cn.pool.ntp.org.',b'0.cn.pool.ntp.org.',172.24.1.81,2.887254e+09,172.24.1.1,2.887254e+09,True,141,50:c7:bf:09:f3:4c,88818833814348,b8:27:eb:2d:24:15,202481588839445,55754.0,53.0,UDP,1.512678e+09,39.713524
118,2017-12-07 15:16:20.096710,b'fr.pool.ntp.org.',None,172.24.1.1,2.887254e+09,172.24.1.81,2.887254e+09,True,75,b8:27:eb:2d:24:15,202481588839445,50:c7:bf:09:f3:4c,88818833814348,53.0,34673.0,UDP,1.512678e+09,295.781260
119,2017-12-07 15:16:20.105092,b'fr.pool.ntp.org.',b'fr.pool.ntp.org.',172.24.1.81,2.887254e+09,172.24.1.1,2.887254e+09,True,513,50:c7:bf:09:f3:4c,88818833814348,b8:27:eb:2d:24:15,202481588839445,34673.0,53.0,UDP,1.512678e+09,295.789642


In [28]:
def num_DNS_queries_and_responses():
    dns_query_packets = filter_data(data, "dns_query", ".notnull()")
    dns_resp_packets = filter_data(data, "dns_resp", ".notnull()")
    count = len(dns_query_packets) + len(dns_resp_packets)
    return count

In [30]:
def dns_query_resp_prob():
    total_packets = len(data)
    dns_packets = num_DNS_queries_and_responses()
    return dns_packets / total_packets
    
dns_query_resp_prob()

0.08695652173913043

Overall, very few of the packets transmitted by the TPLink Router were DNS packets! Why is this the case?

Now, let's introduce conditional probability. Compute the probability that a packet from our capture was a DNS packet, given that its port was 53.

$P($DNS Query $\cup$ DNS Response | Source Port == 53 $\cup$ Dst Port == 53$)$

In [35]:
def num_src_or_dst_port_53():
    port_53_src = filter_data(data, "port_src", "== 53")
    port_53_dst = filter_data(data, "port_dst", "== 53")
    count = len(port_53_src) + len(port_53_dst)
    return count

In [36]:
def dns_query_resp_prob_given_port_53():
    num_query_resp = num_DNS_queries_and_responses()
    num_port = num_src_or_dst_port_53()
    return num_port / num_query_resp

dns_query_resp_prob_given_port_53()

0.6666666666666666

Your answer may surprise you. If DNS packets always contain port 53, why is this probability lower than you'd expect?

Hint: Examine the packets that contain a DNS query or response.

_Answer: DNS queries only contain a query, but DNS respones contain a response AND a query._

Now, fill in the following function that gives the correct conditional probability.

In [46]:
def dns_packets_given_port_53():
    num_DNS_packets = len(filter_data(data, "dns_query", ".notnull()"))
    num_src_port = len(filter_data(data, "port_src", "== 53"))
    num_dst_port = len(filter_data(data, "port_dst", "== 53"))
    return (num_src_port + num_dst_port) / num_DNS_packets

dns_packets_given_port_53()

1.0

Great!

Now let's answer the following questions about our packets. What is the probability that a given DNS response has a length longer than the average length of all packets?

$P($Length > Mean Packet Length | DNS Response$)$

In [62]:
mean_length = data["length"].mean()
DNS_responses = filter_data(data, "dns_resp", ".notnull()")
longer_than_mean = filter_data(DNS_responses, "length", "> " + str(mean_length))
print(longer_than_mean)
return len(longer_than_mean) / mean_length

DNS_response_longer_than_mean()

                      datetime                 dns_query  \
17  2017-12-07 15:11:31.775682  b'devs.tplinkcloud.com.'   
119 2017-12-07 15:16:20.105092       b'fr.pool.ntp.org.'   
123 2017-12-07 15:16:20.364034  b'devs.tplinkcloud.com.'   

                     dns_resp       ip_dst    ip_dst_int      ip_src  \
17   b'devs.tplinkcloud.com.'  172.24.1.81  2.887254e+09  172.24.1.1   
119       b'fr.pool.ntp.org.'  172.24.1.81  2.887254e+09  172.24.1.1   
123  b'devs.tplinkcloud.com.'  172.24.1.81  2.887254e+09  172.24.1.1   

       ip_src_int  is_dns  length            mac_dst     mac_dst_int  \
17   2.887254e+09    True     533  50:c7:bf:09:f3:4c  88818833814348   
119  2.887254e+09    True     513  50:c7:bf:09:f3:4c  88818833814348   
123  2.887254e+09    True     533  50:c7:bf:09:f3:4c  88818833814348   

               mac_src      mac_src_int  port_dst  port_src protocol  \
17   b8:27:eb:2d:24:15  202481588839445   43866.0      53.0      UDP   
119  b8:27:eb:2d:24:15  2024815888394

SyntaxError: 'return' outside function (<ipython-input-62-329105ed9795>, line 5)

It seems that a very small number of DNS responses exceed the average length. Let's examine them.

In [64]:
filter_data(DNS_responses, "length", "> " + str(mean_length))

,datetime,dns_query,dns_resp,ip_dst,ip_dst_int,ip_src,ip_src_int,is_dns,length,mac_dst,mac_dst_int,mac_src,mac_src_int,port_dst,port_src,protocol,time,time_normed
17,2017-12-07 15:11:31.775682,b'devs.tplinkcloud.com.',b'devs.tplinkcloud.com.',172.24.1.81,2.887254e+09,172.24.1.1,2.887254e+09,True,533,50:c7:bf:09:f3:4c,88818833814348,b8:27:eb:2d:24:15,202481588839445,43866.0,53.0,UDP,1.512677e+09,7.460232
119,2017-12-07 15:16:20.105092,b'fr.pool.ntp.org.',b'fr.pool.ntp.org.',172.24.1.81,2.887254e+09,172.24.1.1,2.887254e+09,True,513,50:c7:bf:09:f3:4c,88818833814348,b8:27:eb:2d:24:15,202481588839445,34673.0,53.0,UDP,1.512678e+09,295.789642
123,2017-12-07 15:16:20.364034,b'devs.tplinkcloud.com.',b'devs.tplinkcloud.com.',172.24.1.81,2.887254e+09,172.24.1.1,2.887254e+09,True,533,50:c7:bf:09:f3:4c,88818833814348,b8:27:eb:2d:24:15,202481588839445,59227.0,53.0,UDP,1.512678e+09,296.048584


Next, we'll answer the question: Do long DNS responses take a longer period of time for the server to respond to the request? That is, is the time difference between the request and subsequent response for these packets greater than average?

Later, we will use an annotated dataset of 

# Naive Bayes

Now we're going to use the Naive Bayes algorithm to predict which task a user is most likely doing given a particular packet. While there are existing python functions for performing a naive Bayes classification, we already know everything we need to do it ourselves!

## Data Preprocessing

Like the previous lecture, we first need to label the data with what activity was happening at the time each packet is received.

In [205]:
# Load the data, may take a few minutes.
data = pcap_to_pandas("example_pcaps/ross.pcap")
labels = pd.read_csv('example_pcaps/ross_labels.txt', header=None, names=["time", "activity"])

First, let's add the timestamp (remember, this is measured in seconds since the epoch) to the data set.

In [206]:
def convert_to_datetime(time):
    return datetime.strptime(time, '%Y-%m-%d %H:%M:%S.%f')
    
labels['datetime'] = labels['time'].apply(convert_to_datetime)

tzlocal = datetime.now().astimezone().tzinfo
labels['timestamp'] = labels['datetime'].apply(lambda dt: dt.replace(tzinfo=tzlocal).timestamp())

Next, we're going to use the activity log to label the data set.

In [207]:
label_encoder = LabelEncoder()
labels['label'] = label_encoder.fit_transform(labels['activity'])

for index, row in labels.iterrows():
    data.loc[data['time'] >= row['timestamp'], 'label'] = row['label']
    
num_labels = max(labels['label'])

Finally, we're going to take 20% of the data set and reserve it as test data.

In [208]:
msk = np.random.rand(len(data)) < 0.8
train = data[msk]
test = data[~msk]

## Classification

The simplest statistic we need to compute is the probability that each label occurs:

In [209]:
label_probs = np.zeros(num_classes + 1)

for i in range(num_classes + 1):
    label_probs[i] = len(train[train['label'] == i]) / len(train)

Next, we are going to go through the training set and tally up what values appear in different fields and how often they appear

In [210]:
ip_table = {} # The table stores tuples of the form (label, value)
ip_list = [] # The list stores every value we've seen (uniquely).

dns_table = {}
dns_list = []

port_table = {}
port_list = []

protocol_table = {}
protocol_list = []

def update_table(table, lst, label, value):
    if value is not None:
        table[(label, value)] = table.get((label, value), 0) + 1
    if value is not None and value not in lst:
        lst.append(value)

for index, row in train.iterrows():
    update_table(ip_table, ip_list, row['label'], row['ip_src'])
    update_table(ip_table, ip_list, row['label'], row['ip_dst'])
    update_table(port_table, port_list, row['label'], row['port_src'])
    update_table(port_table, port_list, row['label'], row['port_dst'])
    update_table(protocol_table, protocol_list, row['label'], row['protocol'])
    update_table(dns_table, dns_list, row['label'], row['dns_query'])
    update_table(dns_table, dns_list, row['label'], row['dns_resp'])

Now we use these tallies to compute the logarithm of the event probabilites. Typically, we prefer to work with log probabilities because many of these events have very small chances of happening. When multiplied together the resulting joint probability often end up inconveniently small for computers to work with. Taking logarithms will not change what we are trying to do conceptually, but improves the numerical properties of the algorithm.

In [212]:
def compute_log_probs(table, lst, smoothing):
    log_probs = {}
    
    for l in range(num_labels + 1):
        total = sum([table.get((l, val), 0) for val in lst])
        
        for val in lst:
            if (l, val) in table:
                log_probs[(l, val)] = log(table[(l, val)] + smoothing) - log(total + smoothing * (len(lst) + 1))
        
        log_probs[(l, '<UNK>')] = log(smoothing) - log(total + smoothing * (len(lst) + 1))
    
    return log_probs

ip_log_prob = compute_log_probs(ip_table, ip_list, 1e-5)
dns_log_prob = compute_log_probs(dns_table, dns_list, 1e-5)
port_log_prob = compute_log_probs(port_table, port_list, 1e-5)
protocol_log_prob = compute_log_probs(protocol_table, protocol_list, 1e-5)

Finally, we are ready to create the classifier. When presented with a new row of data, we simply sum all the relevant log probabilities for each class and report the class with the highest log probability.

In [215]:
def get_log_prob(table, val, label):
    return table.get((label, val), table[(label, '<UNK>')])

def classify(row):
    best_label = -1
    best_label_score = float('-Inf')
    
    for l in range(num_labels + 1):
        score = log(class_probs[l])
        
        score = score + get_log_prob(ip_log_prob, row['ip_src'], l)
        score = score + get_log_prob(ip_log_prob, row['ip_dst'], l)
        
        if row['is_dns']:
            if row['dns_query'] is not None:
                score = score + get_log_prob(dns_log_prob, row['dns_query'], l)
            
            if row['dns_resp'] is not None:
                score = score + get_log_prob(dns_log_prob, row['dns_resp'], l)
        
        score = score + get_log_prob(port_log_prob, row['port_src'], l)
        score = score + get_log_prob(port_log_prob, row['port_dst'], l)
        
        score = score + get_log_prob(protocol_log_prob, row['protocol'], l)
        
        if score > best_label_score:
            best_label = l
            best_label_score = score
    
    return best_label

correct = 0

for index, row in test.iterrows():
    if classify(row) == row['label']:
        correct = correct + 1

print('Accuracy: {}'.format(correct / len(test)))

Accuracy: 0.5656517476216976
